In [11]:
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
import json

In [18]:
with open('/raid/nlp/sameer/fever/tfidf_pred_level_1_dev.json') as json_file:
    retrieved_docs = json.load(json_file)

In [ ]:
list(retrieved_docs.items())[0]

In [13]:
from datasets import load_dataset
fever_wiki = load_dataset("fever", 'wiki_pages')
# fever_nli = load_dataset("fever", 'v1.0')

/raid/nlp/sameer/anaconda3/envs/myenv/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for fever contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/fever
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [14]:
from tqdm import tqdm
wiki_id2lines = {}
for dump in tqdm(fever_wiki['wikipedia_pages']):
    if len(dump['text']) != 0:
        wiki_id2lines[dump['id']] = [dump['text'], dump['lines']]

100%|██████████| 5416537/5416537 [02:41<00:00, 33642.43it/s]


In [15]:
import json
with open('/raid/nlp/sameer/fever/data/fever-data/dev.jsonl') as f:
    dev_data = [json.loads(line) for line in f]

with open('/raid/nlp/sameer/fever/data/fever-data/shared_task_test.jsonl') as f:
    test_data = [json.loads(line) for line in f]

In [7]:
import json
with open('/raid/nlp/sameer/fever/data/fever-data/train.jsonl') as f:
    fever_nli = [json.loads(line) for line in f]

In [9]:
data[0]

{'id': 91198,
 'verifiable': 'NOT VERIFIABLE',
 'label': 'NOT ENOUGH INFO',
 'claim': 'Colin Kaepernick became a starting quarterback during the 49ers 63rd season in the National Football League.',
 'evidence': [[[108548, None, None, None]]]}

In [8]:
import re

# gold_docs = []
# id2gold_docs = {}
# id2gold_lines = {}
claim2evi_sent = {}
claim2evi_id = {}
for i in range(0,len(dev_data)):
    if dev_data[i]['label'] == 'NOT ENOUGH INFO':
        continue
    gold = []
    gold_line = []
    claim2evi_sent[dev_data[i]['claim']] = {}
    claim2evi_id[dev_data[i]['claim']] = {}
    for j in range(len(dev_data[i]['evidence'])):
        for k in range(len(dev_data[i]['evidence'][j])):
            
            if dev_data[i]['evidence'][j][k][2] is not None:
                claim2evi_sent[dev_data[i]['claim']][dev_data[i]['evidence'][j][k][2]] = set()
                claim2evi_id[dev_data[i]['claim']][dev_data[i]['evidence'][j][k][2]] = set()
            if dev_data[i]['evidence'][j][k][3] is not None:
                try:
                    text = wiki_id2lines[dev_data[i]['evidence'][j][k][2]][1].split("\n")[data[i]['evidence'][j][k][3]]
                    text = text.replace("\t", " ").strip(",. ")
                    text = re.sub(r'\s+', ' ', text)
                    text =  " ".join(text.split()[1:])
                    claim2evi_sent[dev_data[i]['claim']][dev_data[i]['evidence'][j][k][2]].add(text)
                    claim2evi_id[dev_data[i]['claim']][data[i]['evidence'][j][k][2]].add(dev_data[i]['evidence'][j][k][3])
                except:
                    continue
    # if len(gold):
    #     id2gold_docs[fever_nli[i]['claim']] = set(gold)
    # if len(gold_line):
    #     id2gold_lines[fever_nli[i]['claim']] = set(gold_line)
# gold_docs = [list(set(i)) for i in gold_docs]

In [ ]:
import re

sent2ids = []
lines_corpus = []
corpus_len = len(fever_wiki['wikipedia_pages']['text'][1:])
for i in tqdm(range(0,corpus_len)):
    for line in fever_wiki['wikipedia_pages'][i]['lines'].split("\n"):
        text = line.replace("\t"," ").strip(",. ")
        text = re.sub(r'\s+', ' ', text)
        text = " ".join(text.split()[1:])
        if text.isnumeric()==True or len(text)==0:
             continue
        lines_corpus.append(text)
print(len(lines_corpus))

In [11]:
from collections import OrderedDict

lines_corpus_dup = list(OrderedDict.fromkeys(lines_corpus))

NameError: name 'lines_corpus' is not defined

In [12]:
mapping = {}
for i in range(0,len(lines_corpus_dup)):
    mapping[lines_corpus_dup[i]] = i

In [13]:
data = {"id": list(mapping.values()), "text": list(mapping.keys())}
df = pd.DataFrame(data)
df.reset_index(drop=True, inplace=True)
df.to_csv('index_wiki_sent.tsv', sep="\t",index=False, header=False)

In [16]:
queries_all = []
queries_len = len(fever_nli_hf["train"]["id"])
cnt = 0
for i in range(0,queries_len):
    if fever_nli_hf["train"][i]['evidence_sentence_id'] == -1:
        continue
    else:
        # print(i)
        if fever_nli_hf["train"][i]['claim'] not in queries_all:
            queries_all.append(fever_nli_hf["train"][i]['claim'])
        # cnt += 1

In [17]:
mapping_queries = {}
ids = [i for i in range(0,len(queries_all))]
for i in range(len(ids)):
    mapping_queries[queries_all[i]] = ids[i]
list(mapping_queries.items())[:5]

[('Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.', 0),
 ('Roman Atwood is a content creator.', 1),
 ('History of art includes architecture, dance, sculpture, music, painting, poetry literature, theatre, narrative, film, photography and graphic arts.',
  2),
 ('The Boston Celtics play their home games at TD Garden.', 3),
 ('The Ten Commandments is an epic film.', 4)]

In [10]:
print(list(claim2evi_sent.items())[0])
print("--------------------------------------")
print(list(claim2evi_id.items())[0])

('Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.', {'Nikolaj_Coster-Waldau': {'He then played Detective John Amsterdam in the short-lived Fox television series New Amsterdam -LRB- 2008 -RRB- , as well as appearing as Frank Pike in the 2009 Fox television film Virtuality , originally intended as a pilot . Fox television Fox Broadcasting Company New Amsterdam New Amsterdam (TV series) Virtuality Virtuality (TV series)'}, 'Fox_Broadcasting_Company': {'The Fox Broadcasting Company -LRB- often shortened to Fox and stylized as FOX -RRB- is an American English language commercial broadcast television network that is owned by the Fox Entertainment Group subsidiary of 21st Century Fox . English English language Fox Entertainment Group Fox Entertainment Group 21st Century Fox 21st Century Fox American United States English language English language commercial commercial broadcasting broadcast terrestrial television television network television network'}})
----------------------

In [19]:
mapping_queries = {}
queries_all = list(claim2evi_id.keys())
ids = [i for i in range(0,len(claim2evi_id))]
for i in range(len(ids)):
    mapping_queries[queries_all[i]] = ids[i]
# list(mapping_queries.items())

In [26]:
data = {"id": list(mapping_queries.values()), "text": list(mapping_queries.keys())}
df = pd.DataFrame(data)
df.reset_index(drop=True, inplace=True)
df.to_csv('queries_new_wiki.tsv', sep="\t",index=False, header=False)

In [11]:
import random
random.seed(10)
from sentence_transformers import InputExample

from random import sample 

import re

queries = []
positives = []
negatives = []
train_examples = []

for claim in claim2evi_id:
    for doc_id in claim2evi_id[claim]:
        if doc_id not in wiki_id2lines:
            continue
        lines_original = wiki_id2lines[doc_id][1].split("\n")
        lines = []
        for line in lines_original:
            text = line.replace("\t"," ").strip(",. ")
            text = re.sub(r'\s+', ' ', text)
            text = " ".join(text.split()[1:])
            lines.append(text)
        for pos_id in claim2evi_id[claim][doc_id]:
            pos_line = lines[pos_id]
            lines = [lines[j] for j in range(len(lines)) if len(lines[j])>0 and lines[j].isnumeric()==False and j not in claim2evi_id[claim][doc_id]]
            neg = sample(lines,min(len(lines), 8))
            for m in range(0,len(neg)):
                train_examples.append(InputExample(texts=[claim, pos_line, neg[m]]))

                # negatives.append(neg[m])
                # positives.append(pos_line)
                # queries.append(claim)

In [20]:
print(train_examples[14])

<InputExample> label: 0, texts: Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.; The Fox Broadcasting Company -LRB- often shortened to Fox and stylized as FOX -RRB- is an American English language commercial broadcast television network that is owned by the Fox Entertainment Group subsidiary of 21st Century Fox . English English language Fox Entertainment Group Fox Entertainment Group 21st Century Fox 21st Century Fox American United States English language English language commercial commercial broadcasting broadcast terrestrial television television network television network; Fox and its affiliated companies operate many entertainment channels in international markets , although these do not necessarily air the same programming as the U.S. network


In [ ]:
import random
random.seed(10)
from random import sample 

import re

queries = []
positives = []
negatives = []
for claim in claim2evi_id:
    for doc_id in claim2evi_id[claim]:
        if doc_id not in wiki_id2lines:
            continue
        lines_original = wiki_id2lines[doc_id][1].split("\n")
        lines = []
        for line in lines_original:
            text = line.replace("\t"," ").strip(",. ")
            text = re.sub(r'\s+', ' ', text)
            text = " ".join(text.split()[1:])
            lines.append(text)
        for pos_id in claim2evi_id[claim][doc_id]:
            pos_line = lines[pos_id]
            lines = [lines[j] for j in range(len(lines)) if len(lines[j])>0 and lines[j].isnumeric()==False and j not in claim2evi_id[claim][doc_id]]
            neg = sample(lines,min(len(lines), 8))
            for m in range(0,len(neg)):
                negatives.append(neg[m])
                positives.append(pos_line)
                queries.append(claim)

In [44]:
import random
random.seed(10)
from random import sample 
import re

queries = []
positives = []
negatives = []
sents = []
labels = []
for claim in claim2evi_id:
    total_lines = []
    pos_len = 0
    for doc_id in claim2evi_id[claim]:
        if doc_id not in wiki_id2lines:
            continue
        lines_original = wiki_id2lines[doc_id][1].split("\n")
        lines = []
        for line in lines_original:
            text = line.replace("\t"," ").strip(",. ")
            text = re.sub(r'\s+', ' ', text)
            text = " ".join(text.split()[1:])
            lines.append(text)

        for pos_id in claim2evi_id[claim][doc_id]:
            pos_line = lines[pos_id]
            queries.append(claim)
            sents.append(pos_line)
            labels.append("positive")
            
        pos_len += len(claim2evi_id[claim][doc_id])


        lines = [lines[j] for j in range(len(lines)) if len(lines[j])>0 and lines[j].isnumeric()==False and j not in claim2evi_id[claim][doc_id]]
        total_lines += lines
    if pos_len > 10:
        neg = sample(total_lines,min(len(total_lines), 2*pos_len-pos_len))
    else:
        neg = sample(total_lines,min(len(total_lines), 10-pos_len))

    for m in range(0,len(neg)):
        sents.append(neg[m])
        queries.append(claim)
        labels.append("negative")

In [45]:
queries[0:11]

['Fox 2000 Pictures released the film Soul Food.',
 'Fox 2000 Pictures released the film Soul Food.',
 'Fox 2000 Pictures released the film Soul Food.',
 'Fox 2000 Pictures released the film Soul Food.',
 'Fox 2000 Pictures released the film Soul Food.',
 'Fox 2000 Pictures released the film Soul Food.',
 'Telemundo is a English-language television network.',
 'Telemundo is a English-language television network.',
 'Telemundo is a English-language television network.',
 'Telemundo is a English-language television network.',
 'Telemundo is a English-language television network.']

In [46]:
sents[0:11]

["Soul Food is a 1997 American comedy-drama film produced by Kenneth `` Babyface '' Edmonds , Tracey Edmonds and Robert Teitel and released by Fox 2000 Pictures . Robert Teitel Robert Teitel comedy-drama film comedy-drama film Tracey Edmonds Tracey Edmonds Fox 2000 Pictures Fox 2000 Pictures",
 'In 2015 , it was announced that 20th Century Fox is planning a sequel for film called More Soul Food , written by Tillman , Jr. . 20th Century Fox 20th Century Fox',
 'Featuring an ensemble cast , the film stars Vanessa L. Williams , Vivica A. Fox , Nia Long , Michael Beach , Mekhi Phifer , Jeffrey D. Sams , Irma P. Hall , Gina Ravera and Brandon Hammond . Brandon Hammond Brandon Hammond Vanessa L. Williams Vanessa L. Williams Vivica A. Fox Vivica A. Fox Nia Long Nia Long Michael Beach Michael Beach Mekhi Phifer Mekhi Phifer Jeffrey D. Sams Jeffrey D. Sams Irma P. Hall Irma P. Hall Gina Ravera Gina Ravera ensemble cast ensemble cast',
 'Written and directed by George Tillman , Jr. -- in his maj

In [48]:
labels[0:11]

['positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'positive',
 'negative',
 'negative',
 'negative']

In [49]:
df_data = {"query": queries, "text": sents, "labels": labels}
df = pd.DataFrame(df_data)
df.reset_index(drop=True, inplace=True)
print(df.head())
df.to_csv('all_to_all_interaction_dev.tsv', sep="\t",index=False, header=False)

                                            query  \
0  Fox 2000 Pictures released the film Soul Food.   
1  Fox 2000 Pictures released the film Soul Food.   
2  Fox 2000 Pictures released the film Soul Food.   
3  Fox 2000 Pictures released the film Soul Food.   
4  Fox 2000 Pictures released the film Soul Food.   

                                                text    labels  
0  Soul Food is a 1997 American comedy-drama film...  positive  
1  In 2015 , it was announced that 20th Century F...  negative  
2  Featuring an ensemble cast , the film stars Va...  negative  
3  Written and directed by George Tillman , Jr. -...  negative  
4  Tillman based the family in the film on his ow...  negative  


In [6]:
from random import sample 
import re

queries = []
positive = []
negatives = []
for i in tqdm(range(queries_len)):
    if fever_nli["train"][i]["evidence_sentence_id"] == -1:
        continue
    else:
        try:
            gold_doc_id = fever_nli["train"][i]['evidence_wiki_url']
            gold_sent_id = fever_nli["train"][i]['evidence_sentence_id']
            lines_original = wiki_id2lines[gold_doc_id].split("\n")
            lines = []
            for line in lines_original:
                text = line.replace("\t"," ").strip(",. ")
                text = re.sub(r'\s+', ' ', text)
                lines.append(text)
        except:
            continue
        pos = lines[fever_nli["train"][i]['evidence_sentence_id']]
        # pos = re.sub(r'\s+', ' ', pos)
        lines = [lines[j] for j in range(len(lines)) if len(lines[j])>0 and lines[j].isnumeric()==False and j!=gold_sent_id]
        # temp = [mapping_queries[fever_nli["train"][i]["claim"]], mapping[pos]]
        neg = sample(lines,min(len(lines), 8))

        for m in range(0,min(len(lines), 8)):
            queries.append(mapping_queries[fever_nli["train"][i]["claim"]])
            positive.append(mapping[pos])
            negatives.append(mapping[neg[m]])
        # for m in range(0,len(neg)):
        #     negatives.append(neg[m])
        # negatives += neg

NameError: name 'queries_len' is not defined

In [25]:
import json

with open('fever_original_triples_1_pos_8_neg.json', 'w') as triples_file:
    for i in range(0,len(negatives)):
        temp = []
        triples_file.write(json.dumps([queries[i], positives[i], negatives[i]])  + '\n')

In [39]:
queries

[0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2]

In [40]:
positive

[16510508,
 16510508,
 16510508,
 19750164,
 19750164,
 19750164,
 19750166,
 19750166,
 19750166,
 10318844,
 10318844,
 10318844]

In [43]:
negatives

[16510505,
 16510506,
 16510510,
 19750163,
 19750167,
 19750168,
 19750165,
 19750164,
 19750163,
 10318849,
 10318848,
 10318847]

In [9]:
for i in tqdm(range(10)):
    if fever_nli["train"][i]["evidence_sentence_id"] == -1:
        continue
    else:
        print(fever_nli["train"][i])
        print("-------------------------")

100%|██████████| 10/10 [00:00<00:00, 6611.45it/s]

{'id': 75397, 'label': 'SUPPORTS', 'claim': 'Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.', 'evidence_annotation_id': 92206, 'evidence_id': 104971, 'evidence_wiki_url': 'Nikolaj_Coster-Waldau', 'evidence_sentence_id': 7}
-------------------------
{'id': 150448, 'label': 'SUPPORTS', 'claim': 'Roman Atwood is a content creator.', 'evidence_annotation_id': 174271, 'evidence_id': 187498, 'evidence_wiki_url': 'Roman_Atwood', 'evidence_sentence_id': 1}
-------------------------
{'id': 150448, 'label': 'SUPPORTS', 'claim': 'Roman Atwood is a content creator.', 'evidence_annotation_id': 174271, 'evidence_id': 187499, 'evidence_wiki_url': 'Roman_Atwood', 'evidence_sentence_id': 3}
-------------------------
{'id': 214861, 'label': 'SUPPORTS', 'claim': 'History of art includes architecture, dance, sculpture, music, painting, poetry literature, theatre, narrative, film, photography and graphic arts.', 'evidence_annotation_id': 255136, 'evidence_id': 254645, 'evidence_wiki_url': 

In [8]:
fever_nli["train"][0]

{'id': 75397,
 'label': 'SUPPORTS',
 'claim': 'Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.',
 'evidence_annotation_id': 92206,
 'evidence_id': 104971,
 'evidence_wiki_url': 'Nikolaj_Coster-Waldau',
 'evidence_sentence_id': 7}

In [16]:
def return_line_corpus(pred_docs_ids):
    line_corpus = []
    wiki2lines = {}
    for doc_id in pred_docs_ids:
        try:
            lines = wiki_id2lines[doc_id][1].split("\n")
        except:
            continue
        for i in range(len(lines)):
            text = lines[i].replace("\t"," ").strip(",. ")
            text = re.sub(r'\s+', ' ', text)
            text = " ".join(text.split()[1:])
            wiki2lines[text] = [doc_id, i]
            if text.isnumeric()==True or len(text)==0:
                continue
            line_corpus.append(text)
    return(line_corpus, wiki2lines)

In [ ]:
from rank_bm25 import BM25Okapi
import re

queries = []
bm25_lines = []
for sample in tqdm(dev_data):
    line_corpus, wiki2lines = return_line_corpus(retrieved_docs[sample['claim']])
    tokenized_corpus = [doc.split(" ") for doc in line_corpus]

    bm25 = BM25Okapi(tokenized_corpus)
    query = sample['claim']
    tokenized_query = query.split(" ")

    retrieved_lines = bm25.get_top_n(tokenized_query, line_corpus, n=50)
    for line in retrieved_lines:
        queries.append(sample['claim'])
        bm25_lines.append(wiki2lines[line])

In [21]:
len(bm25_lines)==len(queries)

True

In [22]:
df_data = {"query": queries, "text": bm25_lines, "labels": ["negative" for i in range(len(bm25_lines))]}
df = pd.DataFrame(df_data)
df.reset_index(drop=True, inplace=True)
print(df.head())
df.to_csv('all_to_all_interaction_dev_bm25.tsv', sep="\t",index=False, header=False)

                                               query  \
0  Colin Kaepernick became a starting quarterback...   
1  Colin Kaepernick became a starting quarterback...   
2  Colin Kaepernick became a starting quarterback...   
3  Colin Kaepernick became a starting quarterback...   
4  Colin Kaepernick became a starting quarterback...   

                                                text    labels  
0  season was notable when Colin Kaepernick sat d...  negative  
1  Wolf Pack also became the first team in colleg...  negative  
2  49ers finished 30th in passing yards per game ...  negative  
3  pistol has also made the transition to the NFL...  negative  
4  to 2002 , the 49ers led the all-time series 4 ...  negative  
